# SQL pré-armazenado: stored procedures, triggers, views

Além de armazenar nossos dados em tabelas, os bancos de dados também podem armazenar código SQL na forma de vários tipos de objetos:

- **Stored procedures**: procedimentos escritos em SQL, executados através da chamada `CALL`.
- **Stored functions**: funções escritas em SQL e que podem ser usadas nas mesmas situações que uma função pré-definida seria usada, como `SUM()` ou `COUNT()`
- **Triggers**: Um procedimento que será executado automaticamente quando determinadas condições ocorrem, como `INSERT`, `UPDATE` ou `DELETE` em uma tabela.
- **Eventos**: Procedimentos que podem ser executados pelo banco de dados em horários pré-definidos.
- **Views**: Diferente dos outros objetos, uma view é como um `SELECT` pré-definido, e resulta em uma tabela virtual.

Vamos continuar trabalhando com a base de dados **emprestimo** de uma aula anterior.

<img src="img/diagrama.png">

Para facilitar a atividade de hoje, vamos começar do zero rodando o script `emprestimos.sql`:

```SQL
DROP DATABASE IF EXISTS emprestimos;
CREATE DATABASE emprestimos;
USE emprestimos;

CREATE TABLE usuario (
    id_usuario INT NOT NULL AUTO_INCREMENT,
    nome VARCHAR(80) NOT NULL,
    sobrenome VARCHAR(80) NOT NULL,
    saldo DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    PRIMARY KEY (id_usuario),
    CONSTRAINT c_saldo CHECK (saldo >= 0.0)
);

CREATE TABLE emprestimo (
    id_emprestimo INT NOT NULL AUTO_INCREMENT,
    id_credor INT NOT NULL,
    id_devedor INT NOT NULL,
    valor_atual DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    data_inicio DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    data_modificação DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (id_emprestimo),
    CONSTRAINT fk_credor FOREIGN KEY (id_credor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT fk_devedor FOREIGN KEY (id_devedor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT c_valor CHECK (valor_atual >= 0.0)
);

CREATE TABLE operacao (
    id_operacao INT NOT NULL AUTO_INCREMENT,
    id_emprestimo INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_operacao),
    CONSTRAINT fk_emprestimo FOREIGN KEY (id_emprestimo)
        REFERENCES emprestimo (id_emprestimo)
);

CREATE TABLE movimentacao (
    id_movimentacao INT NOT NULL AUTO_INCREMENT,
    id_usuario INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_movimentacao),
    CONSTRAINT fk_usuario FOREIGN KEY (id_usuario)
        REFERENCES usuario (id_usuario)
);
```

Em seguida, vamos abrir a conexão com o banco de dados.

In [1]:
import mysql.connector
from functools import partial
import os
import insperautograder.jupyter as ia
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
script = """
DROP DATABASE IF EXISTS emprestimos;
CREATE DATABASE emprestimos;
USE emprestimos;

CREATE TABLE usuario (
    id_usuario INT NOT NULL AUTO_INCREMENT,
    nome VARCHAR(80) NOT NULL,
    sobrenome VARCHAR(80) NOT NULL,
    saldo DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    PRIMARY KEY (id_usuario),
    CONSTRAINT c_saldo CHECK (saldo >= 0.0)
);

CREATE TABLE emprestimo (
    id_emprestimo INT NOT NULL AUTO_INCREMENT,
    id_credor INT NOT NULL,
    id_devedor INT NOT NULL,
    valor_atual DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    data_inicio DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    data_modificação DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (id_emprestimo),
    CONSTRAINT fk_credor FOREIGN KEY (id_credor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT fk_devedor FOREIGN KEY (id_devedor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT c_valor CHECK (valor_atual >= 0.0)
);

CREATE TABLE operacao (
    id_operacao INT NOT NULL AUTO_INCREMENT,
    id_emprestimo INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_operacao),
    CONSTRAINT fk_emprestimo FOREIGN KEY (id_emprestimo)
        REFERENCES emprestimo (id_emprestimo)
);

CREATE TABLE movimentacao (
    id_movimentacao INT NOT NULL AUTO_INCREMENT,
    id_usuario INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_movimentacao),
    CONSTRAINT fk_usuario FOREIGN KEY (id_usuario)
        REFERENCES usuario (id_usuario)
);
"""

# connection = mysql.connector.connect(
#     host=os.getenv('MD_DB_SERVER'),
#     user=os.getenv('MD_DB_USERNAME'),
#     password=os.getenv('MD_DB_PASSWORD'),
# )

# with connection.cursor() as cursor:
#     try:
#         for _ in cursor.execute(script, multi=True):
#             pass
#         cursor.execute("COMMIT")
#     except Exception as e:
#         print(e)
#         cursor.execute("ROLLBACK")

# connection.close()

In [3]:
def get_connection_helper(database):

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            # print("Executando queries:")
            results = cursor.execute(query, args, multi=True)
            for i, result in enumerate(results):
                if result.with_rows:
                    print(f"Resultado query {i}:")
                    for line in result.fetchall():
                        print(line)
                else:
                    print(f"Query {i} executada!")

    connection = mysql.connector.connect(
        host=os.getenv("MD_DB_SERVER"),
        user=os.getenv("MD_DB_USERNAME"),
        password=os.getenv("MD_DB_PASSWORD"),
        database=database,
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper("emprestimos")

## Stored procedures

Existem algumas vantagens em procedimentos armazenados (stored procedures):

- **Reuso de código**: como em qualquer outra linguagem de programação, podemos construir stored procedures para definir tarefas rotineiras e reutilizá-las em vários workflows.
- **Segurança**: Ao invés de oferecer acesso direto à tabelas, podemos bloquear todos os acessos a tabelas e definir acesso à stored procedures! Assim, ao invés de permitir acesso à tabela *usuario*, podemos permitir acesso ao procedimento `saque()` ou `consulta_saldo()`
- **Desempenho**: ao invés de mandar vários comandos SQL para a base de dados toda vez que quisermos realizar determinada tarefa, podemos simplesmente chamar um procedimento armazenado, pois todo o código já está na base de dados. Ademais, a base de dados não precisa compilar o procedimento toda vez que este é chamado, basta compilar na primeira execução e manter o código compilado em um cache.

Existem também desvantagens:

- **Debugging**: pode ser difícil debugar um procedimento armazenado
- **Portabilidade**: a sintaxe de definição de stored procedures é raramente portável entre diferentes sistemas de gerenciamento de bancos de dados.
- **Separação entre dados e lógica de negócios**: com stored procedures estamos migrando parte da lógica de negócios para o banco de dados. Alterações na lógica agora demandam modificações (e manutenção) em partes distintas da sua aplicação, uma receita para dor de cabeça!
- **Aumento da carga de processamento no servidor**: servidores de banco de dados costumam ser otimizados para memória e largura de banda, não para processamento.

Ou seja use procedimentos armazenados com cautela!

### Interlúdio: o valor da experiência

Como você já deve ter percebido à essa altura do curso, você tem um camilho longo pela frente, desde conhecer um conceito novo até dominar o uso deste conceito com sabedoria - isso chama-se ganhar experiência! Não subestime o valor de trabalhar "nas trincheiras" aplicando seus conhecimentos novos e aprendendo com a experiência dos veteranos. Além disso, continue se aperfeiçoando - faça cursos para conhecer melhor a tecnologia, invente projetos para tentar novas ideias, assista vídeos no YouTube sobre melhores práticas, busque projetos (e empregos) onde você possa se desenvolver.

É impossível formar engenheiros experientes em um curso - mesmo que fossem dois cursos, não cobriríamos tudo que existe sobre banco de dados. Mas estou certo de que vocês adquiriram novas ferramentas neste módulo do curso, e estão capacitados a continuar progredindo em banco de dados.

Nós, os professores do Insper, estamos fazendo o possível para transferir a vocês mais do que conhecimento - estamos buscando desenvolver competência também! E para isso só existe um caminho: a prática. Façam exercícios, coloquem esforço nos projetos, e venham preparados para a aula para que possamos ir além da mera leitura do material didático, ok?

Voltamos agora à programação regular...

### Construindo uma stored procedure

Vamos começar com uma stored procedure para adicionar um usuário. Crie e execute o seguinte script no MySQL workbench:

```SQL
USE emprestimos;

DROP PROCEDURE IF EXISTS adiciona_usuario;

DELIMITER //
CREATE PROCEDURE adiciona_usuario(IN novo_nome VARCHAR(80), IN novo_sobrenome VARCHAR(80))
BEGIN
    INSERT INTO usuario (nome, sobrenome)
    VALUES (novo_nome, novo_sobrenome);
END//
DELIMITER ;
```

Note o uso destes comandos `DELIMITER`. Coisas de MySQL: ele não entende que os ponto-e-virgula internos ao procedimento não sinalizam o final do comando CREATE PROCEDURE... Este procedimento será necessário quando quiser criar as procedures no Workbench, mas não deverá ser utilizado quando quiser rodar direto do python (com mysql.connector podemos utilizar ponto e vírgula).

Chamando pelo Python ficará assim:

```SQL
USE emprestimos;

DROP PROCEDURE IF EXISTS adiciona_usuario;

CREATE PROCEDURE adiciona_usuario(IN novo_nome VARCHAR(80), IN novo_sobrenome VARCHAR(80))
BEGIN
    INSERT INTO usuario (nome, sobrenome)
    VALUES (novo_nome, novo_sobrenome);
END;
```

Agora podemos usar nossa nova procedure para adicionar alguns usuários!

In [20]:
db('SELECT * FROM usuario')

Resultado query 0:
(7, 'alemagno', 'santos', Decimal('68999.90'))
(8, 'Cock', 'G', Decimal('12999.90'))
(9, 'Shao', 'Ming', Decimal('14999.90'))


In [21]:
db("COMMIT");

Query 0 executada!


In [22]:
db("START TRANSACTION;")
try:
    db("CALL adiciona_usuario('Juca', 'Silva');")
    db("CALL adiciona_usuario('Mario', 'Ferreira');")
    db("CALL adiciona_usuario('Ana', 'Soares');")
    db("CALL adiciona_usuario('Antonio', 'Reis');")
    db("CALL adiciona_usuario('Paulo', 'Oliveira');")
    db("COMMIT;")
except Exception as e:
    print(e)
    db("ROLLBACK");

Query 0 executada!
1305 (42000): PROCEDURE emprestimos.adiciona_usuario does not exist
Query 0 executada!


Vamos verificar se tudo funcionou bem:

In [23]:
db("SELECT * FROM usuario")

Resultado query 0:
(7, 'alemagno', 'santos', Decimal('68999.90'))
(8, 'Cock', 'G', Decimal('12999.90'))
(9, 'Shao', 'Ming', Decimal('14999.90'))


### Stored functions

Agora vamos criar uma função para consultar o saldo. Rode o script a seguir no MySQL Workbench:

```SQL
USE emprestimos;

DROP FUNCTION IF EXISTS saldo;

DELIMITER //
CREATE FUNCTION saldo(id INT) RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN
    DECLARE saldo_procurado DECIMAL(30, 2);

    SELECT 
        IFNULL(saldo, 0.0)
    INTO saldo_procurado FROM
        usuario
    WHERE
        id_usuario = id;

    RETURN saldo_procurado;
END//
DELIMITER ;
```

Vamos testar esta função informando o id de usuário `1`:

In [25]:
db("""USE emprestimos;

DROP FUNCTION IF EXISTS saldo;

CREATE FUNCTION saldo(id INT) RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN
    DECLARE saldo_procurado DECIMAL(30, 2);

    SELECT 
        IFNULL(saldo, 0.0)
    INTO saldo_procurado FROM
        usuario
    WHERE
        id_usuario = id;

    RETURN saldo_procurado;
END""")

Query 0 executada!
Query 1 executada!
Query 2 executada!


In [27]:
db("SELECT saldo(7)")

Resultado query 0:
(Decimal('68999.90'),)


### Exercícios para entrega

Esta aula tem atividade para entrega, confira os prazos e exercícios

In [11]:
ia.tasks()

|    | Atividade            | De                        | Até                       |
|---:|:---------------------|:--------------------------|:--------------------------|
|  0 | newborn              | 2024-02-01 03:00:00+00:00 | 2024-05-30 03:00:00+00:00 |
|  1 | select01             | 2024-02-08 03:00:00+00:00 | 2024-02-19 02:59:59+00:00 |
|  2 | ddl                  | 2024-02-22 03:00:00+00:00 | 2024-02-27 02:59:59+00:00 |
|  3 | dml                  | 2024-02-26 03:00:00+00:00 | 2024-03-03 02:59:59+00:00 |
|  4 | group_having         | 2024-02-29 03:00:00+00:00 | 2024-03-12 02:59:59+00:00 |
|  5 | views                | 2024-02-29 03:00:00+00:00 | 2024-03-20 02:59:59+00:00 |
|  6 | agg_join             | 2024-02-29 03:00:00+00:00 | 2024-03-05 02:59:59+00:00 |
|  7 | sql_review1          | 2024-03-11 03:00:00+00:00 | 2024-03-20 02:59:59+00:00 |
|  8 | permissions          | 2024-03-18 03:00:00+00:00 | 2024-03-26 02:59:59+00:00 |
|  9 | desafio_normalizacao | 2024-03-21 03:00:00+00:00 | 2024-04-15 02:59:59+00:00 |
| 10 | ai_md_23_1           | 2024-03-25 03:00:00+00:00 | 2024-04-01 15:00:00+00:00 |
| 11 | ai_md_23_2           | 2024-03-25 03:00:00+00:00 | 2024-04-01 15:00:00+00:00 |
| 12 | ai_md_24_1           | 2024-04-01 03:00:00+00:00 | 2024-04-01 18:35:00+00:00 |
| 13 | triggers             | 2024-04-18 03:00:00+00:00 | 2024-04-26 02:59:59+00:00 |

In [12]:
ia.grades(task="triggers")

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | triggers    | ex01        |      1 |      0 |
|  1 | triggers    | ex02        |      1 |      0 |
|  2 | triggers    | ex04        |      1 |      0 |
|  3 | triggers    | ex05        |      1 |      0 |
|  4 | triggers    | ex06        |      1 |      0 |
|  5 | triggers    | ex07        |      1 |      0 |
|  6 | triggers    | ex08        |      1 |      0 |
|  7 | triggers    | ex09        |      1 |      0 |

In [13]:
ia.grades(by="task", task="triggers")

|    | Tarefa   |   Nota |
|---:|:---------|-------:|
|  0 | triggers |      0 |

**<div id="perg_atv1"></div>**
### Exercícios

<div class="alert alert-info">
    
**OBS**: Alteramos a função `db()` para aceitar o envio de múltiplas queries na mesma string!

</div>

#### Exercício 1
Faça um script SQL que cria uma stored procedure para cobrar uma taxa de manutenção de `D` dinheiros de cada conta, desde que a conta tenha saldo (não cobra de quem nã otem saldo suficiente para descontar a taxa).

**Dica**: use a função `IF()`: https://dev.mysql.com/doc/refman/8.0/en/flow-control-functions.html#function_if

<div>

Faça a partir do código base:

```SQL
USE emprestimos;

DROP PROCEDURE IF EXISTS cobra_taxa;

CREATE PROCEDURE cobra_taxa(IN taxa DECIMAL(30, 2))
BEGIN
    --- Gambiarra para desligar o aviso de segurança que previne
    --- UPDATES gerais (sem WHERE).
    -- OBS: da pra fazer sem ela!
    SET @OLD_SQL_SAFE_UPDATES = @@SQL_SAFE_UPDATES;
    SET SQL_SAFE_UPDATES=0;

    --- Insira seu código aqui.
    
    --- Desliga a gambiarra.
    SET SQL_SAFE_UPDATES=@OLD_SQL_SAFE_UPDATES;
END;
```   

</div>

In [16]:
sql_ex01 = """
USE emprestimos;

DROP PROCEDURE IF EXISTS cobra_taxa;

CREATE PROCEDURE cobra_taxa(IN taxa DECIMAL(30, 2))
BEGIN
    SET @OLD_SQL_SAFE_UPDATES = @@SQL_SAFE_UPDATES;
    SET SQL_SAFE_UPDATES=0;

    UPDATE usuario
        SET saldo = IF(saldo > taxa, saldo-taxa, saldo);

    SET SQL_SAFE_UPDATES=@OLD_SQL_SAFE_UPDATES;
END;
"""

db(sql_ex01)

Query 0 executada!
Query 1 executada!
Query 2 executada!


Escreva um código SQL para testar sua procedure:

**Resposta:**

In [17]:
db("CALL cobra_taxa(0.1);")

Query 0 executada!


In [ ]:
ia.sender(answer="sql_ex01", task="triggers", question="ex01", answer_type="pyvar")

<a href="#gab_atv1">Click para ver a resposta</a>

**<div id="perg_atv2"></div>**
#### Exercício 2

Faça uma função que retorna a soma de todos os saldos da base de dados.

<div>

```SQL
USE emprestimos;

DROP FUNCTION IF EXISTS total_saldos;

CREATE FUNCTION total_saldos() RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN

    -- Insira seu SQL aqui.

END;
```

</div>

In [18]:
sql_ex02 = """
USE emprestimos;

DROP FUNCTION IF EXISTS total_saldos;

CREATE FUNCTION total_saldos() RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN
    DECLARE total DECIMAL(30,2);
    SELECT SUM(saldo) INTO total FROM usuario;
    RETURN total;
END;
"""

db(sql_ex02)

Query 0 executada!
Query 1 executada!
Query 2 executada!


Escreva um código SQL para testar sua função:

**Resposta:**

In [19]:
db("SELECT total_saldos() AS TotalSaldos;")

Resultado query 0:
(Decimal('96999.70'),)


In [ ]:
ia.sender(answer="sql_ex02", task="triggers", question="ex02", answer_type="pyvar")

<a href="#gab_atv2">Click para ver a resposta</a>

## Triggers

*Triggers*, ou gatilhos, são procedimentos armazenados que são executados automaticamente quando uma operação é realizada. Você pode escolher se o trigger ocorre antes ou depois da ação.

Vamos fazer um gatilho para atualizar automaticamente o saldo do usuário quando uma nova movimentação é inserida na tabela `movimentacao`. Vamos também criar uma *constraint* para impedir que um saldo negativo exista na nossa base de dados.

Bom, rode o script SQL a seguir no MySQL Workbench:

```SQL
USE emprestimos;

DROP TRIGGER IF EXISTS trig_movimentacao;

DELIMITER //
CREATE TRIGGER trig_movimentacao 
BEFORE INSERT ON movimentacao
FOR EACH ROW
BEGIN
    UPDATE usuario 
        SET saldo = saldo + NEW.valor 
        WHERE id_usuario = NEW.id_usuario;
END//

DELIMITER ;
```

*Interlúdio*: e quanto ao constraint `CHECK`? No MySQL 5.7 o `CHECK` não funcionava, apesar de previsto no padrão SQL. Na versão 8.0 isso foi corrigido.

Na documentação do MySQL 5.7 (https://dev.mysql.com/doc/refman/5.7/en/create-table.html) temos o seguinte:

    The `CHECK` clause is parsed but ignored by all storage engines. See Section 1.8.2.3, “Foreign Key Differences”.

A título de curiosidade: o remédio era criar um trigger para evitar a atualização com valores errados. Veja a gambiarra em ação no código a seguir.

```SQL
USE emprestimos;

DROP TRIGGER IF EXISTS trig_saldo_insuficiente;

DELIMITER //
CREATE TRIGGER trig_saldo_insuficiente 
BEFORE UPDATE ON usuario
FOR EACH ROW
BEGIN
    -- https://stackoverflow.com/questions/2115497/check-constraint-in-mysql-is-not-working
    IF NEW.saldo < 0.0 THEN
        SIGNAL SQLSTATE '12345'
            SET MESSAGE_TEXT = 'Saldo insuficiente.';
    END IF;
END//
DELIMITER ;
```

Já na versão atual do MySQL o check de saldo pode ser implementado normalmente com o constraint `CHECK`

```SQL
USE emprestimos;
ALTER TABLE usuario ADD CONSTRAINT chk_saldo CHECK (saldo >= 0.0);
ALTER TABLE emprestimo ADD CONSTRAINT chk_emprestimo CHECK (valor_atual >= 0.0);
```

Vamos testar esse trigger na nossa tabela:

In [30]:
db("""
USE emprestimos;

DROP TRIGGER IF EXISTS trig_movimentacao;
CREATE TRIGGER trig_movimentacao 
BEFORE INSERT ON movimentacao
FOR EACH ROW
BEGIN
    UPDATE usuario 
        SET saldo = saldo + NEW.valor 
        WHERE id_usuario = NEW.id_usuario;
END;
   """)

Query 0 executada!
Query 1 executada!
Query 2 executada!


In [31]:
db("""
USE emprestimos;
ALTER TABLE usuario ADD CONSTRAINT chk_saldo CHECK (saldo >= 0.0);
ALTER TABLE emprestimo ADD CONSTRAINT chk_emprestimo CHECK (valor_atual >= 0.0);
""")

Query 0 executada!
Query 1 executada!
Query 2 executada!


In [33]:
def movimentacao(id_usuario, valor):
    db(
        "INSERT INTO movimentacao (id_usuario, valor) VALUES (%s, %s)",
        (id_usuario, valor),
    )
    db("SELECT * FROM usuario WHERE id_usuario=%s", (id_usuario,))

In [34]:
db("START TRANSACTION")
movimentacao(7, 1000)

Query 0 executada!
Query 0 executada!
Resultado query 0:
(7, 'alemagno', 'santos', Decimal('69999.90'))


In [35]:
db('SELECT * FROM movimentacao')

Resultado query 0:
(1, 7, Decimal('1000.00'), datetime.datetime(2024, 4, 21, 12, 37, 56))


In [36]:
db('SELECT * FROM usuario')

Resultado query 0:
(7, 'alemagno', 'santos', Decimal('69999.90'))
(8, 'Cock', 'G', Decimal('12999.90'))
(9, 'Shao', 'Ming', Decimal('14999.90'))


Parece ter funcionado... mas antes de executar o `COMMIT`, vamos testar o que acontece quando o saldo ficaria negativo:

In [38]:
movimentacao(8, -200)

Query 0 executada!
Resultado query 0:
(8, 'Cock', 'G', Decimal('12799.90'))


Opa, o constraint impediu o saque inválido! Excelente! Vamos reverter as ultimas mudanças antes de prosseguir:

In [66]:
db("ROLLBACK")

Query 0 executada!


### Exercício 3

Realize as seguintes movimentações (com COMMIT se tudo funcionou dessa vez, e ROLLBACK se falhar):

| id_usuario | valor |
|--|--|
| 1 | +10000 |
| 2 | +4000 |
| 3 | +15000 |

In [39]:
movimentacao(7, 1)
movimentacao(8, 4000)
movimentacao(9, 15000)

Query 0 executada!
Resultado query 0:
(7, 'alemagno', 'santos', Decimal('70000.90'))
Query 0 executada!
Resultado query 0:
(8, 'Cock', 'G', Decimal('16799.90'))
Query 0 executada!
Resultado query 0:
(9, 'Shao', 'Ming', Decimal('29999.90'))


**Conferir resposta**:

In [40]:
db("SELECT * FROM movimentacao")

Resultado query 0:
(1, 7, Decimal('1000.00'), datetime.datetime(2024, 4, 21, 12, 37, 56))
(2, 8, Decimal('-200.00'), datetime.datetime(2024, 4, 21, 12, 38, 30))
(3, 7, Decimal('1.00'), datetime.datetime(2024, 4, 21, 12, 39))
(4, 8, Decimal('4000.00'), datetime.datetime(2024, 4, 21, 12, 39))
(5, 9, Decimal('15000.00'), datetime.datetime(2024, 4, 21, 12, 39))


In [41]:
db("SELECT * FROM usuario")

Resultado query 0:
(7, 'alemagno', 'santos', Decimal('70000.90'))
(8, 'Cock', 'G', Decimal('16799.90'))
(9, 'Shao', 'Ming', Decimal('29999.90'))


### Exercício 4

Crie um trigger chamada `trig_transferencias_emprestimo` para realizar operações de transferência no contexto de um empréstimo.

In [42]:
sql_ex04 = """
use emprestimos;

DROP TRIGGER IF EXISTS trig_transferencias_emprestimo;

CREATE TRIGGER trig_transferencias_emprestimo
BEFORE INSERT ON operacao
FOR EACH ROW
BEGIN
    DECLARE credor int;
    DECLARE devedor int;

    SELECT id_credor, id_devedor 
    INTO credor, devedor
    FROM emprestimo
    WHERE id_emprestimo = NEW.id_emprestimo;

    UPDATE emprestimo
        SET valor_atual = valor_atual + NEW.valor
    WHERE id_emprestimo = NEW.id_emprestimo;

    INSERT INTO movimentacao (id_usuario, valor) 
        VALUES 
            (credor, NEW.valor * (-1)),
            (devedor, NEW.valor);
END;
"""

db(sql_ex04)

Query 0 executada!
Query 1 executada!
Query 2 executada!


In [ ]:
ia.sender(answer="sql_ex04", task="triggers", question="ex04", answer_type="pyvar")

**Conferir resposta**:

In [5]:
db("SELECT * FROM usuario")

Resultado query 0:


In [43]:
db("INSERT INTO emprestimo (id_credor, id_devedor) VALUES (2, 3)")

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`emprestimos`.`emprestimo`, CONSTRAINT `fk_credor` FOREIGN KEY (`id_credor`) REFERENCES `usuario` (`id_usuario`))

In [ ]:
db("SELECT * FROM emprestimo")

In [ ]:
db("SELECT * FROM operacao")

In [ ]:
db("INSERT INTO operacao (id_emprestimo, valor) VALUES (1, 2000)")

In [ ]:
db("SELECT * FROM operacao")

In [ ]:
db("SELECT * FROM emprestimo")

In [ ]:
db("SELECT * FROM usuario")

In [ ]:
db("SELECT * FROM movimentacao")

## Relembrando: Views

*Views* são como tabelas *'lógicas'*, que são criadas através da aplicação de um comando `SELECT`. É como um `SELECT` pré-armazenado.

Vamos criar uma *view* para listar os nomes e sobrenomes de usuários, sem revelar seus saldos de conta:

```SQL
USE emprestimos;

CREATE VIEW nomes AS 
	SELECT DISTINCT nome, sobrenome FROM usuario;
```

Agora podemos usar esta view em consultas:

In [13]:
db("""INSERT INTO usuario (nome,sobrenome,saldo) VALUES 
	("alemagno", "santos", 69000),
    ("Cock", "G", 13000),
    ("Shao", "Ming", 15000);""")

Query 0 executada!


In [14]:
db("SELECT * FROM nomes;")

Resultado query 0:
('alemagno', 'santos')
('Cock', 'G')
('Shao', 'Ming')


Porque usar views? Assim como no caso de stored procedures, podemos configurar permissões de acesso diferentes para esta view. Suponha que um vendedor deva ter acesso aos nomes dos clientes, mas não aos seus saldos (por razões de confidencialidade). Podemos conceder ao vendedor acesso apenas à essa view. Poderíamos ter resolvido o problema também com uma stored procedure: em SQL as coisas costumam ter várias soluções possíveis...

### Exercício 5

Crie uma view chamada `credor` para mostrar, para cada usuário, a soma dos valores de emprestimo dos quais este cliente é credor. Se o cliente não tiver empréstimo associado como credor, o valor deve ser zero.

As colunas retornadas devem se chamar `id_usuario` e `total`, onde `total` representa a soma dos valores de empréstimo dos quais o cliente é credor.

Ordene pelo `total` (decrescente), nome (crescente) e sobrenome (crescente).

In [51]:
sql_ex05 = """
DROP VIEw IF EXISTS credor;
CREATE VIEW credor AS 
    SELECT id_usuario, nome, sobrenome, COALESCE(SUM(valor_atual),0) AS total FROM usuario AS u
    LEFT OUTER JOIN emprestimo e ON u.id_usuario = e.id_credor
    GROUP BY id_usuario
    ORDER BY total DESC, nome, sobrenome DESC
"""

db(sql_ex05)

Query 0 executada!
Query 1 executada!


In [45]:
ia.sender(answer="sql_ex05", task="triggers", question="ex05", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex05', style=ButtonStyle()), Output()), _dom_classes=('widget…

### Exercício 6

Repita o processo para a posição de devedor. A view deve se chamar `devedor`.

Retorne uma coluna `total_liquido`.Considere os mesmos nomes de colunas e critérios de ordenação.

In [59]:
sql_ex06 = """
DROP VIEw IF EXISTS devedor;
CREATE VIEW devedor AS 
    SELECT id_usuario, nome, sobrenome, COALESCE(SUM(valor_atual),0) AS total_liquido FROM usuario AS u
    LEFT OUTER JOIN emprestimo e ON u.id_usuario = e.id_devedor
    GROUP BY id_usuario
    ORDER BY total_liquido DESC, nome, sobrenome DESC
"""

db(sql_ex06)

Query 0 executada!
Query 1 executada!


In [54]:
ia.sender(answer="sql_ex06", task="triggers", question="ex06", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex06', style=ButtonStyle()), Output()), _dom_classes=('widget…

### Exercício 7

Crie uma view `posicao_liquida` que lista o **valor líquido** de cada usuário, que é a soma dos valores como credor mais o saldo, subtraido da soma dos valores como devedor.

Considere os mesmos nomes de colunas e critérios de ordenação.

In [ ]:
sql_ex07 = """
DROP VIEW IF EXISTS posicao_liquida;
CREATE VIEW posicao_liquida AS 
    SELECT 
        u.id_usuario, 
        u.nome, 
        u.sobrenome, 
        COALESCE(u.saldo+c.total-d.total,0) AS total_liquido
    FROM usuario AS u
    JOIN credor c USING(id_usuario)
    JOIN devedor d USING(id_usuario)
    GROUP BY id_usuario
    ORDER BY total_liquido DESC, nome, sobrenome DESC;
"""

db(sql_ex07)

In [67]:
db("INSERT INTO emprestimo (id_credor,id_devedor,valor_atual) VALUES (7,8,500);")

Query 0 executada!


In [70]:
db("SELECT * FROM posicao_liquida;")

Resultado query 0:
(7, 'alemagno', 'santos', Decimal('69500.90'))
(9, 'Shao', 'Ming', Decimal('29999.90'))
(8, 'Cock', 'G', Decimal('17299.90'))


In [72]:
db("SELECT * FROM devedor;")

Resultado query 0:
(8, 'Cock', 'G', Decimal('500.00'))
(7, 'alemagno', 'santos', Decimal('0.00'))
(9, 'Shao', 'Ming', Decimal('0.00'))


In [63]:
ia.sender(answer="sql_ex07", task="triggers", question="ex07", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex07', style=ButtonStyle()), Output()), _dom_classes=('widget…

### Exercício 8

Crie uma função para calcular o preço com desconto.

A função se chama `aplica_desconto` e recebe, nesta ordem, o `preco DECIMAL(10,2)` e o `percentual DECIMAL(10,2)` de desconto (intervalo `[0,100]`).

**Dicas**:
- procure por *"MySQL functions DETERMINISTIC, NO SQL, READS SQL DATA"*.
- você pode criá-la em qualquer base de dados (não tem relação com a base `emprestimos`).

In [76]:
sql_ex08 = """
CREATE FUNCTION aplica_desconto(preco DECIMAL(10,2), percentual DECIMAL(10,2))
RETURNS DECIMAL(10,2) READS SQL DATA
BEGIN
    RETURN preco * (1 - (percentual*0.01));
END;
"""

db(sql_ex08)

Query 0 executada!


In [77]:
ia.sender(answer="sql_ex08", task="triggers", question="ex08", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex08', style=ButtonStyle()), Output()), _dom_classes=('widget…

### Exercício 9

Considere uma base chamada de **clientes** e **solicitações** chamada `solicita`:

<img src="img/solicita.png">

Crie uma **procedure** que deleta **todos os dados** dos clientes cujo último acesso foi há mais que 180 dias.

A procedure deve se chamar `remove_clientes_inativos`.

**Dicas**:
- Como critério de último acesso, considere a coluna `ultimo_acesso` na tabela de `cliente`.
- Utilize os scripts SQL disponíveis na pasta de aulas para inserir clientes e solicitações na base e efetuar os testes!
- Altere as datas, pois os 180 dias são em relação a data atual.

In [95]:
db("""
UPDATE solicita.cliente
SET ultimo_acesso = DATE_SUB(CURDATE(), INTERVAL 181 DAY)
WHERE id_cliente IN (1, 3, 5);

UPDATE solicita.cliente
SET ultimo_acesso = DATE_SUB(CURDATE(), INTERVAL 90 DAY)
WHERE id_cliente IN (2, 4);
""")

Query 0 executada!
Query 1 executada!


In [111]:
sql_ex09 = """
DROP PROCEDURE IF EXISTS remove_clientes_inativos;
CREATE PROCEDURE remove_clientes_inativos()
BEGIN
    DELETE FROM solicitacao
    WHERE id_cliente IN (
        SELECT id_cliente FROM cliente
        WHERE ultimo_acesso < DATE_SUB(CURDATE(), INTERVAL 180 DAY)
    );

    DELETE FROM cliente 
    WHERE ultimo_acesso < DATE_SUB(CURDATE(), INTERVAL 180 DAY);
END;     
"""

In [97]:
ia.sender(answer="sql_ex09", task="triggers", question="ex09", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex09', style=ButtonStyle()), Output()), _dom_classes=('widget…

### Exercício 10

Este exercício não tem correção automática e sua resolução é opcional.

Considere uma versão base de **clientes** e **solicitações** onde os deletes são **lógicos** e não físicos:

<img src="img/solicita_v2.png">

Neste exercício, você deve:

- Criar procedure para **deletar logicamente clientes** cujo último acesso foi há mais que 180 dias. A procedure deve deletar logicamente apenas clientes e não solicitações.
- Criar trigger na tabela de cliente para que, ao deletar logicamente um cliente, suas solicitações também sejam deletadas.
- Criar trigger na tabela de solicitação para impedir a inserção de solicitações para cliente deletados.

`ex10:`

In [ ]:
p0 = """
ALTER TABLE cliente
    ADD COLUMN (
        ativo TINYINT DEFAULT 1
    );
"""

In [ ]:
p1 = """
DROP PROCEDURE IF EXISTS remove_clientes_inativos;
CREATE PROCEDURE remove_clientes_inativos()
BEGIN
    UPDATE cliete
    SET ativo = 0
    WHERE id_cliente IN (
        SELECT id_cliente FROM cliente
        WHERE ultimo_acesso < DATE_SUB(CURDATE(), INTERVAL 180 DAY)
    );

    DELETE FROM solicitacao
    WHERE id_cliente IN (
        SELECT id_cliente FROM cliente
        WHERE ativo = 0;
    );
END;
"""

In [ ]:
p2 = """"
DROP TRIGGER IF EXISTS remove_solicitacoes;
CREATE TRIGGER remove_solicitacoes
AFTER UPDATE ON cliente
BEGIN
    IF NEW.ativo = 0 THEN
        DELETE FROM solicitacao
        WHERE id_cliente = NEW.id_cliente
    END IF;
END; 
"""

In [ ]:
p3 = """
DROP TRIGGER IF EXISTS ActiveCheck;
CREATE TRIGGER ActiveCheck 
BEFORE INSERT ON solicitacao
FOR EACH ROW
BEGIN 
    DECLARE status TINYINT;

    SELECT ativo INTO status
    FROM cliente AS c
    WHERE c.id_cliente = NEW.id_cliente

    IF status = 0 THEN 
        SIGNAL SQLSTATE '45000' 
        SET MESSAGE_TEXT = 'Não é possível inserir solicitação para cliente inativo.';
    END IF;
END;
"""

Por hoje é só!

In [ ]:
connection.close()

### Conferir notas

In [112]:
ia.grades(by="task", task="triggers")

|    | Tarefa   |   Nota |
|---:|:---------|-------:|
|  0 | triggers |     10 |

In [113]:
ia.grades(task="triggers")

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | triggers    | ex01        |      1 |     10 |
|  1 | triggers    | ex02        |      1 |     10 |
|  2 | triggers    | ex04        |      1 |     10 |
|  3 | triggers    | ex05        |      1 |     10 |
|  4 | triggers    | ex06        |      1 |     10 |
|  5 | triggers    | ex07        |      1 |     10 |
|  6 | triggers    | ex08        |      1 |     10 |
|  7 | triggers    | ex09        |      1 |     10 |

## Gabarito

**<div id="gab_atv1">Atividade 1</div>**
<div class="alert alert-warning">

Para Workbench:

```SQL
USE emprestimos;

DROP PROCEDURE IF EXISTS cobra_taxa;

DELIMITER //
CREATE PROCEDURE cobra_taxa(IN taxa DECIMAL(30, 2))
BEGIN
    -- Gambiarra para desligar o aviso de segurança que previne
    -- UPDATES gerais (sem WHERE).
    -- OBS: da pra fazer sem ela!
    SET @OLD_SQL_SAFE_UPDATES = @@SQL_SAFE_UPDATES;
    SET SQL_SAFE_UPDATES=0;

    -- Insira seu código aqui.
    UPDATE usuario 
    SET 
        saldo = saldo - taxa
    WHERE
        saldo >= taxa;
    
    -- Desliga a gambiarra.
    SET SQL_SAFE_UPDATES=@OLD_SQL_SAFE_UPDATES;
END //
DELIMITER ;
```

Para Python:

```SQL
USE emprestimos;

DROP PROCEDURE IF EXISTS cobra_taxa;

CREATE PROCEDURE cobra_taxa(IN taxa DECIMAL(30, 2))
BEGIN
    -- Gambiarra para desligar o aviso de segurança que previne
    -- UPDATES gerais (sem WHERE).
    -- OBS: da pra fazer sem ela!
    SET @OLD_SQL_SAFE_UPDATES = @@SQL_SAFE_UPDATES;
    SET SQL_SAFE_UPDATES=0;

    -- Insira seu código aqui.
    UPDATE usuario 
    SET 
        saldo = saldo - taxa
    WHERE
        saldo >= taxa;
    
    -- Desliga a gambiarra.
    SET SQL_SAFE_UPDATES=@OLD_SQL_SAFE_UPDATES;
END;
```    
</div>
<a href="#perg_atv1">Click para voltar ao Exercício</a>


**<div id="gab_atv2">Atividade 2</div>**
<div class="alert alert-warning">

Para Workbench:
```SQL
USE emprestimos;

DROP FUNCTION IF EXISTS total_saldos;

DELIMITER //
CREATE FUNCTION total_saldos() RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN

    -- Insira seu SQL aqui.
    DECLARE total DECIMAL(30, 2);
    SELECT SUM(saldo) INTO total FROM usuario;
    RETURN total;

END//
DELIMITER ;
```
</div>
<a href="#perg_atv2">Click para voltar ao Exercício</a>